In [1]:
using POMDPs, POMDPTools, QuickPOMDPs, MCTS, DiscreteValueIteration, POMDPSimulators, POMDPModels, POMDPPolicies
using Distributions, Combinatorics, StaticArrays, D3Trees, Random
using FileIO, JLD2, TickTock

In [2]:
action=[(0,0), (0,1), (1,1)]

Number_level = 10;
#Transition_matrix = [0.86 0.14 0.0 0.0; 0.0 0.83 0.17 0.0;0.0 0.0 0.89 0.11;1.0 0.0 0.0 0.0];

failure_penalty = -1000.0;
maintenance_penalty = -800.0;
setup_cost = -300.0;
normal_operation = 45.0;

In [3]:
function reward(s, a,k,s1=10, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair
    p=[sum(1-T[i,i,1,j] for i in 1:9) for j in 1:NumberUnits]/sum(sum(1-T[i,i,1,j] for i in 1:9) for j in 1:NumberUnits)   
    if s==Number_level
            r=failure_penalty  + normal_operation  ;
        return r
       end
        r=0
     if a==1
        if s1>=s
            crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r = normal_operation ;
         return r
        else
            return 0.0
        end
        end
    if a==2
        if s==1
            crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r= normal_operation   ;
            return r
        else
        s1=deepcopy(s)
        s=1
        r = maintenance_penalty 
         crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r+= normal_operation  ;
#            r*=(s1/10);
        return r
        end
    end
    end

reward (generic function with 3 methods)

In [4]:
s=1:Number_level;

In [5]:
T=zeros(Number_level,Number_level,2,100);

In [6]:
for i in 1:100
    fullname = "TM_MATRIX/tm"*string(i);
    # generate_transition_matrix(Number_level, fullname) 
    Transition_matrix = load(fullname*".jld2","transition_matrix");
    
    T[:,:,1,i].=Transition_matrix;
    T[:,:,2,i].=Transition_matrix;
    for j in 1:10
        T[j,:,2,i].=Transition_matrix[1,:];
        end
    T[10,:,1,i].=Transition_matrix[1,:]
T[10,:,2,i].=Transition_matrix[1,:]
end

In [7]:
T

10×10×2×100 Array{Float64, 4}:
[:, :, 1, 1] =
 0.232649  0.229122  0.130585  0.0954644  …  0.0507595  0.0416229  0.00861626
 0.0       0.218724  0.163234  0.128353      0.0899794  0.0431818  0.00868773
 0.0       0.0       0.166395  0.182476      0.111027   0.0773355  0.043199
 0.0       0.0       0.0       0.269557      0.125296   0.0981466  0.0533485
 0.0       0.0       0.0       0.0           0.125828   0.109077   0.0611228
 0.0       0.0       0.0       0.0        …  0.215491   0.17866    0.0619123
 0.0       0.0       0.0       0.0           0.352355   0.221208   0.0646198
 0.0       0.0       0.0       0.0           0.471891   0.449917   0.0781922
 0.0       0.0       0.0       0.0           0.0        0.591116   0.408884
 0.232649  0.229122  0.130585  0.0954644     0.0507595  0.0416229  0.00861626

[:, :, 2, 1] =
 0.232649  0.229122  0.130585  0.0954644  …  0.0507595  0.0416229  0.00861626
 0.232649  0.229122  0.130585  0.0954644     0.0507595  0.0416229  0.00861626
 0.232649  

In [8]:
function explore()
    c=[1/3,1/3,1/3]

     crd = Categorical(c);
     return rand(crd)
    end 
function generative(s, a,k, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair
        if s==Number_level
        s=1
        crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r=failure_penalty  + normal_operation  ;
        return (sp=s, r=r)
       end
        r=0
     if a==1
            crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r = normal_operation ;
         return (sp=s, r=r)
        end
    if a==2
#         if s==1
#             crd = Categorical(T[s,:,1,k]);
#             s = rand(crd);
#             r= normal_operation+  setup_cost/NumberUnits  ;
#             return (sp=s, r=r)
#         else
        s1=deepcopy(s)
        s=1
        r = maintenance_penalty 
         crd = Categorical(T[s,:,1,k]);
            s = rand(crd);
            r+= normal_operation  ;
#            r*=(s1/10);
       return (sp=s, r=r)
#         end
    end
    end

generative (generic function with 2 methods)

In [9]:
function simulate!(V,a,T, s,S,γ, d)
    if d ≤ 0
        return sum(T[s,s1,1]*V[s1] for (s1) in S)
    end
#     a = explore()
#     print("IN")

    s′, r = generative(s,a)
    q = r + γ*simulate!(V,a,T, s′,S,γ, d-1)
#     print("out")
#     N[s,a] += 1
#     Q[s,a] += (q-Q[s,a])/N[s,a]
    return q
end

simulate! (generic function with 1 method)

In [10]:
function ValueIteration(S, T, γ, p1=0.01, d=100)
    Q=zeros((Number_level,2,NumberUnits))
#     print(Q)
    V= rand(Float64, (Number_level,NumberUnits))*1000
    diff=0
    i=0
    
    

    
    
   Threads.@threads for k in 1:NumberUnits
        p=[1 1 1 ]
    for i in 1:1500
#     while  i<=10000
    for s in 1:Number_level
        for a in 1:2
                Q[s,a,k]= (reward(s,a,k)) + γ*( sum(T[s,s1,a,k]*(V[s1,k]) for (s1) in 1:Number_level ))
        end
       V[s,k]=findmax(Q[s,1:2,k])[1]

        end
        
        end
    end
#     println(Q)
    return Q
    end

ValueIteration (generic function with 3 methods)

In [11]:
# function ValueIteration(S, T, γ, p1=0.01, d=100)
#     Q=zeros((Number_level,3,NumberUnits)).+100
# #     print(Q)
#     V= rand(Float64, (Number_level,NumberUnits))*1000
#     V1= rand(Float64, (Number_level,NumberUnits))*1000

#     diff=0
#     i=0
    
    

    
#      p=zeros(Float64, (Number_level,NumberUnits,3)).+(1/3)
#     for k in 1:NumberUnits
       
#     for i in 1:500
# #     while  i<=10000
#     for s in 1:Number_level
#         Threads.@threads for a in 1:3
#                 Q[s,a,k]= (reward(s,a,k)) + γ*( sum(T[s,s1,a,k]*(V[s1,k]) for (s1) in 1:Number_level ))
#         end
#             p[s,k,:]=[exp(lamb*sum(Q[s,1,k])),exp(lamb*sum(Q[s,2,k])),exp(lamb*sum(Q[s,3,k]))]
#             p[s,k,:]=p[s,k,:]./sum(p[s,k,:])
# #        V[s,k]=Q[s,1,k]*p[s,k,1]+max(Q[s,2,k],Q[s,3,k])*(p[s,k,2]+p[s,k,3])
#        V[s,k]=Q[s,1,k]*p[s,k,1]+max(Q[s,2,k],Q[s,3,k])*(p[s,k,2]+p[s,k,3])
# #         for s in 1:Number_level
# #         p1=[exp(lamb1*V[s,k]) for k in 1:NumberUnits]
# #         p1=p1./sum(p1)
# #         V[s,:].=sum([p1[k]*V[s,k] for k in 1:NumberUnits])
# # #         V[s,:].=min(V[s,:]...)#/NumberUnits
# #         end
#         end
# #         for s in 1:Number_level
# # #         p1=[exp(lamb1*V1[s,k]) for k in 1:NumberUnits]
# # #         p1=p1./sum(p1)
# # #         V[s,:].=sum([p1[k]*V1[s,k] for k in 1:NumberUnits])
# #         V[s,:].=mean(V1[s,:])#/NumberUnits
# #         end
#         end
#     end
#     return Q
#     end

In [12]:
 Q=ValueIteration(s,T,.95,0.01,3000)

LoadError: UndefVarError: NumberUnits not defined

In [13]:
global lamb=.1 
Q=ValueIteration(s,T,.95,1,1000)

LoadError: UndefVarError: NumberUnits not defined

In [14]:
s

1:10

In [15]:

# function decison(Q, s)
#    a_final=[]
#    if Number_level in s
# #         print("hi")
#         d=(collect(Iterators.product((Q[s[i],2:3] for i in 1:NumberUnits)...)))
#         d1=(collect(Iterators.product((2:3 for i in 1:NumberUnits)...)))
#         min=-1e+10
#         opt=0
#         for (i,j) in enumerate(d)
#             if min < sum((j))
#                 min=sum((j))
#                 opt=i
#             end
#         end
#         return d1[opt]
#     end
#     q0=0.0
#     a_0=repeat(1:1,NumberUnits);
#     for i in 1:NumberUnits
#             q0+=Q[s[i],1]
#     end
#     d=(collect(Iterators.product((Q[s[i],2:3] for i in 1:NumberUnits)...)))
#     d1=(collect(Iterators.product((2:3 for i in 1:NumberUnits)...)))
# #     d[1]=a_0
# #     print("here")
# #     d1[1]=(1,1,1,1,1,1,1,1,1,1)
#     min=-1e+10
#     opt=0
#     for (i,j) in enumerate(d)
#         if i==1
#             if min < q0
#             min=q0
#             opt=i
#         end
#         elseif min < sum((j))
#             min=sum((j))
#             opt=i
#         end
#     end
#     if opt==1
#         return a_0
#     else
#         return d1[opt]
#     end
#     return opt
# end


function decison(Q, s,n=0.2)
 a_final=[]
#         print("hi")
        for i in 1:NumberUnits
#         println/(i)
            append!(a_final,argmax(a->Q[s[i],a,i], [1,2]))
            end
        return a_final
end

decison (generic function with 2 methods)

In [16]:
decison(Q,[8,8
        ,8,8
        ,1,1,6,1, 7,7,8,8
        ,8,8
        ,1,1,6,1, 7,7])

LoadError: UndefVarError: Q not defined

In [17]:
Q

LoadError: UndefVarError: Q not defined

In [18]:
s

1:10

In [19]:
using DataFrames
df= DataFrame(Units=[],n=[],s=[],m=[],f=[],lambda=[],mean=[],std=[])

Row,Units,n,s,m,f,lambda,mean,std
,Any,Any,Any,Any,Any,Any,Any,Any


In [20]:
cost=[ 
    [0 -1000 -200 -1000],
    [0 -1200 -200 -1000],
    [0 -1000 -200 -1400],
    [0 -1000 -200 -1600]
    
    ]

# print(Q)

for units in [60,50,40,30,20]
    for c in cost
   print(units)  
    global normal_operation,setup_cost,maintenance_penalty,failure_penalty=c
     println([normal_operation,setup_cost,maintenance_penalty,failure_penalty])   
     for lambda in 1:1:1
        global  lamb=lambda
    global NumberUnits=units
     Q=ValueIteration(s,T,.95,3000)
#          println(isnan(Q[1,1]))
#        if sum(isnan(Q[1,1]))>0.0
#             continue
#         end
tick()
trials=10000;
simsteps = 100;
results=zeros(trials)*0.1
Threads.@threads for h in 1:trials
s=repeat(1:1,NumberUnits)
r1=[]
a1=[]   
r=repeat(1:1,NumberUnits)*1.0
 for k in 1:simsteps
    a =decison(Q,s)
    r2=0.0
    s1=repeat(1:1,NumberUnits)
              
            
#     println(a)
    for i in 1:NumberUnits
#             print(generative(s[i],a[i],i))
         s1[i],r[i]=generative(s[i],a[i],i);
          r2+=r[i]
            end 
#     println(r2)
#     println(s)
    if (Number_level in s) || (2 in a)
            r2+=setup_cost
                    end
    s=s1;
    append!(a1,a)
    append!(r1,r2)
end
rk=0
for i in 1:simsteps
    rk=0.95*rk+r1[simsteps-i+1]
end
#     println(rk)
    results[h]=rk
#     println(results)
end
println(units)
println(lambda)

println(mean(results))

println(std(results))
pushfirst!(df,[units,normal_operation,setup_cost,maintenance_penalty,failure_penalty,lamb,mean(results),std(results)])
        
tock()
end
    println("-------------------------")
end
end

60[0, -1000, -200, -1000]


[ Info:  started timer at: 2023-04-02T14:08:57.636


60
1
-104004.80613544049
3747.0020955483433
-------------------------
60[0, -1200, -200, -1000]


[ Info:           93.3068463s: 1 minute, 33 seconds, 306 milliseconds
[ Info:  started timer at: 2023-04-02T14:15:23.692


60
1
-107839.62189725235
3737.347210558693
-------------------------

...

[ Info:           27.9988228s: 27 seconds, 998 milliseconds


, -1000, -200, -1600]


[ Info:  started timer at: 2023-04-02T15:13:03.113


20
1
-55390.2108401892
3095.2940479918007
-------------------------


[ Info:           30.7592844s: 30 seconds, 759 milliseconds


In [21]:
df

Row,Units,n,s,m,f,lambda,mean,std
,Any,Any,Any,Any,Any,Any,Any,Any
1,20.0,0.0,-1000.0,-200.0,-1600.0,1.0,-55390.2,3095.29
2,20.0,0.0,-1000.0,-200.0,-1400.0,1.0,-53296.1,2916.54
3,20.0,0.0,-1200.0,-200.0,-1000.0,1.0,-52100.2,2263.58
4,20.0,0.0,-1000.0,-200.0,-1000.0,1.0,-48378.7,2265.46
5,30.0,0.0,-1000.0,-200.0,-1600.0,1.0,-73521.2,3711.18
6,30.0,0.0,-1000.0,-200.0,-1400.0,1.0,-70415.8,3388.27
7,30.0,0.0,-1200.0,-200.0,-1000.0,1.0,-67135.3,2702.71
8,30.0,0.0,-1000.0,-200.0,-1000.0,1.0,-63384.3,2678.86
9,40.0,0.0,-1000.0,-200.0,-1600.0,1.0,-89282.2,4277.61


In [22]:
using CSV
CSV.write("independent with setup DISJOINT.csv", df)

"independent with setup DISJOINT.csv"

In [23]:
 Q=ValueIteration(s,T,.95,3000)
for s in 1:Number_level
p=[exp(0.01*sum(Q[s,1])),exp(0.01*sum(Q[s,2])),exp(0.01*sum(Q[s,3]))]
    p./=sum(p)
    println(p)
end

LoadError: BoundsError: attempt to access 10×2×20 Array{Float64, 3} at index [1, 1]

In [24]:
df[df.Units.==15,:]

Row,Units,n,s,m,f,lambda,mean,std
,Any,Any,Any,Any,Any,Any,Any,Any


In [25]:
df1= DataFrame(Units=[],mean=[])
for i in [2,4,5,8,10,15,20,25,30]
    pushfirst!(df1,[i,findmax(df[df.Units.==i,:].mean)[1]])
end
CSV.write("valueiteration_softmax_space_division_FINAL.csv", df1)

LoadError: MethodError: reducing over an empty collection is not allowed; consider supplying `init` to the reducer

In [26]:
df1

Row,Units,mean
,Any,Any
